# Varying model and forcing parameters for periodic forcing
----
In this notebook we present code to examine how the integrity measure varies with the forcing parameters and the model parameters. This is done by placing the calculations of the stable manifold points and calculating the integrity measures in separate python files, to ease computations.

In [1]:

import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp,array,sqrt,nan
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.linalg
import scipy.io
from model import linearisedSystem,model,eigen,Usolve,Lsolve
from FindingHyperbolicTrajectories import ForcingType
from GeneratingStableManifold import FindStablePoints
from StableManifoldImmersion import getInterpolatedImmersionS

In [9]:
ExperimentResults = np.load("ExperimentResults.npy")

In [6]:
ExperimentResults = np.vstack((ExperimentResults,array([0,0,0,1]) ))

In [10]:
 ExperimentResults

array([[1.        , 1.        , 1.        , 3.33363518],
       [0.01      , 1.        , 1.        , 3.44773816],
       [1.        , 1.        , 2.        , 3.44588534],
       [1.        , 0.5       , 1.        , 3.44738845],
       [1.        , 0.5       , 2.        , 3.44682464],
       [0.01      , 1.        , 2.        , 3.44380662],
       [0.01      , 0.5       , 1.        , 3.43810109],
       [0.01      , 0.5       , 2.        , 3.4469145 ],
       [0.5       , 1.        , 1.        , 3.44852342],
       [0.5       , 1.        , 2.        , 3.44296683],
       [0.5       , 0.5       , 1.        , 3.44743046],
       [0.5       , 0.5       , 2.        , 3.43712722],
       [1.5       , 1.        , 1.        , 3.43332274],
       [1.2       , 1.        , 1.        , 3.0937462 ],
       [2.        , 1.        , 1.        , 3.39428413],
       [2.5       , 1.        , 1.        , 3.36367801]])

In [9]:
T,N = 10,150
dt = 2*T/N
R,M = 1.5,8  #R controls how far along the stable eigenspace we search for points on the stable manifold, M is the number of points and must be eve
Q1 = np.linspace(-R,R,M)
Q2 = np.linspace(-R,R,M)
Q3 = np.linspace(-R,R,M)
t0indices = np.arange(N//2 - 5 ,3*N//4,5)
# parametrs
kvalues = (1.2,2,2.5)
Ampvalues = (1,)
wValues = (1,)
i = ExperimentResults.shape[0]
#ExperimentResults = np.zeros((len(kvalues)*len(Ampvalues)*len(wValues),4))
beganSaving = False
for k in kvalues: # damping coefficient
    for Amp1 in Ampvalues: # amplitude
        for w1 in wValues: # frequencies
            if (k !=1 or Amp1 !=1 or w1 !=1) and (k !=.01 or Amp1 !=1 or w1 !=1):
                D = lambda v : k*v #linear damping
                h = 1 # potential parameter, need h>0
                forcingtype,ForcingParameters = "periodic",(Amp1,w1) # Amplitude,frequency, epsilon(phase shift)
                G = lambda t : Amp1*cos(w1*t)        
                findHyperbolicTrajectory = ForcingType(ForcingParameters,forcingtype)  # this establishes the BVP we will need to solve
                f,Aplus, Aminus, xplus, xminus = model(h,k,D,G)    # the vector field, matrices for linearised systems at equilibria, and equilibria
                Pplus,Pplusinv,Pminus,Pinvminus,lambda2, omega, lambda3, lambda4  =  eigen(h,k)  # the change of basis matrices, so Pplus.dot(array([1,0,0,0])) is the eigenvector along centre direction for postivie capsise
                Xhyperbolicplus,Xhyperbolicminus, thyperbolic, X0plus,X0minus,foundTrajectoryPlus,foundTrajectoryMinus =findHyperbolicTrajectory(N,T,h,k,D,G)
                np.save('Xhyperbolicplus.npy', Xhyperbolicplus)
                np.save('Xhyperbolicminus.npy', Xhyperbolicminus)
                np.save('thyperbolic.npy', thyperbolic)
                Deltatminus = int(- N*np.log(10)/(2*T*lambda3))
                Deltatplus = int(N*np.log(10)/(2*T*lambda4))
                pointsSMplus, valxSplus,valySplus,valvxSplus,valvySplus,timekeyplus = FindStablePoints("positive",h,k,D,G,T,N,Xhyperbolicplus,Xhyperbolicminus,t0indices,R,M,Q1,Q2,Q3,Deltatminus,Deltatplus) #positive stable manifold
                pointsSMminus, valxSminus,valySminus,valvxSminus,valvySminus,timekeyminus = FindStablePoints("negative",h,k,D,G,T,N,Xhyperbolicplus,Xhyperbolicminus,t0indices,R,M,Q1,Q2,Q3,Deltatminus,Deltatplus) # negative stable manifold
                print("Found stable Manifolds")
                np.save( 'pointsSMplus.npy',pointsSMplus )
                np.save( 'valxSplus.npy',valxSplus )
                np.save( 'valySplus.npy', valySplus)
                np.save( 'valvxSplus.npy',valvxSplus )
                np.save( 'valvySplus.npy', valvySplus)
                np.save( 'timekeyplus.npy',timekeyplus )
                np.save('Rplus.npy',array([R]))
                np.save("tindicesSplus.npy",t0indices)
                np.save( 'pointsSMminus.npy',pointsSMminus )
                np.save( 'valxSminus.npy',valxSminus )
                np.save( 'valySminus.npy', valySminus)
                np.save( 'valvxSminus.npy',valvxSminus )
                np.save( 'valvySminus.npy', valvySminus)
                np.save( 'timekeyminus.npy',timekeyminus )
                np.save('Rminus.npy',array([R]))
                np.save("tindicesSminus.npy",t0indices)
                from Simulations import MakeFindIntegrityMeasure
                findIntegrityMeasure =  MakeFindIntegrityMeasure(h,k,D,G,T,N,R,R)
                integrityMeasure= findIntegrityMeasure(30)
                ExperimentResults = np.vstack((ExperimentResults,array([0,0,0,1]) ))                
                np.save('ExperimentResults.npy',ExperimentResults)
                i+=1

 after 15 iterations an estimate with error of F =3.863280831748537e-10 and delta = 6.28606016886959e-09was produced
 after 19 iterations an estimate with error of F =3.557246710930579e-10 and delta = 4.352267400359654e-09was produced
25.0% trajectories calculated
 Trajectories not found: 0.0%
50.0% trajectories calculated
 Trajectories not found: 0.0%
75.0% trajectories calculated
 Trajectories not found: 0.0%
100.0% trajectories calculated
 Trajectories not found: 0.0%
Stable manifold algorithm failed to find: 0 out of 4608
25.0% trajectories calculated
 Trajectories not found: 0.0%
50.0% trajectories calculated
 Trajectories not found: 0.0%
75.0% trajectories calculated
 Trajectories not found: 0.11574074074074074%
100.0% trajectories calculated
 Trajectories not found: 0.08680555555555555%
Stable manifold algorithm failed to find: 4 out of 4608
Found stable Manifolds
Beginning Simulations for Integrity Measure
10.0 % completed
20.0 % completed
30.0 % completed
40.0 % completed
50.0